In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [187]:
data = pd.read_csv('pulse_eco_one_sensor_data.csv')
data.head(5)

,Type,Value,Stamp
0,humidity,87,2023-01-01T01:06:45+01:00
1,humidity,87,2023-01-01T01:22:18+01:00
2,humidity,87,2023-01-01T01:37:50+01:00
3,humidity,87,2023-01-01T01:53:21+01:00
4,humidity,87,2023-01-01T02:08:53+01:00


In [188]:
data['Stamp']=pd.to_datetime(data['Stamp'],utc=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307207 entries, 0 to 307206
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype              
---  ------  --------------   -----              
 0   Type    307207 non-null  object             
 1   Value   307207 non-null  int64              
 2   Stamp   307207 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(1), object(1)
memory usage: 7.0+ MB


In [189]:
#najbliski 15 mins
data['Stamp'] = data['Stamp'].dt.floor('15min')

In [190]:
data.head(5)

,Type,Value,Stamp
0,humidity,87,2023-01-01 00:00:00+00:00
1,humidity,87,2023-01-01 00:15:00+00:00
2,humidity,87,2023-01-01 00:30:00+00:00
3,humidity,87,2023-01-01 00:45:00+00:00
4,humidity,87,2023-01-01 01:00:00+00:00


In [191]:
#gi sortira samiot

In [192]:
data.sort_values('Stamp')
data.head(10)

,Type,Value,Stamp
0,humidity,87,2023-01-01 00:00:00+00:00
1,humidity,87,2023-01-01 00:15:00+00:00
2,humidity,87,2023-01-01 00:30:00+00:00
3,humidity,87,2023-01-01 00:45:00+00:00
4,humidity,87,2023-01-01 01:00:00+00:00
5,humidity,87,2023-01-01 01:15:00+00:00
6,humidity,87,2023-01-01 01:30:00+00:00
7,humidity,87,2023-01-01 01:45:00+00:00
8,humidity,87,2023-01-01 02:00:00+00:00
9,humidity,86,2023-01-01 02:15:00+00:00


In [193]:
data['Type']

0            humidity
1            humidity
2            humidity
3            humidity
4            humidity
             ...     
307202    temperature
307203    temperature
307204    temperature
307205    temperature
307206    temperature
Name: Type, Length: 307207, dtype: object

In [194]:
pivot = data.pivot_table(index='Stamp',columns='Type',values='Value',aggfunc='mean')
pivot = pivot.resample('15min').first()

In [195]:
pivot.head(5)

Type,humidity,noise_dba,pm10,pm25,temperature
Stamp,,,,,
2023-01-01 00:00:00+00:00,87.0,45.0,473.0,156.0,5.0
2023-01-01 00:15:00+00:00,87.0,44.0,333.0,120.0,5.0
2023-01-01 00:30:00+00:00,87.0,46.0,328.0,108.0,5.0
2023-01-01 00:45:00+00:00,87.0,44.0,415.0,125.0,5.0
2023-01-01 01:00:00+00:00,87.0,44.0,341.0,110.0,5.0


In [196]:
pivot.isnull().sum()

Type
humidity       5010
noise_dba      3804
pm10           5317
pm25           5312
temperature    5010
dtype: int64

In [197]:
#ne mora gornata linija kod so floor zatoa sto istoto go pravi i resample

In [198]:
#pravime matrica za da vidime dali treba interpolacija/ffill/bfill

In [199]:
pivot.interpolate(method='cubic', limit_direction='both', inplace=True)

In [200]:
pivot.isnull().sum()

Type
humidity       0
noise_dba      0
pm10           0
pm25           0
temperature    0
dtype: int64

In [201]:
pivot.head(5)

Type,humidity,noise_dba,pm10,pm25,temperature
Stamp,,,,,
2023-01-01 00:00:00+00:00,87.0,45.0,473.0,156.0,5.0
2023-01-01 00:15:00+00:00,87.0,44.0,333.0,120.0,5.0
2023-01-01 00:30:00+00:00,87.0,46.0,328.0,108.0,5.0
2023-01-01 00:45:00+00:00,87.0,44.0,415.0,125.0,5.0
2023-01-01 01:00:00+00:00,87.0,44.0,341.0,110.0,5.0


In [202]:
#drop the column noise_dba (is not relevant)
pivot.drop('noise_dba',axis=1,inplace=True)

In [203]:
pivot.head(5)

Type,humidity,pm10,pm25,temperature
Stamp,,,,
2023-01-01 00:00:00+00:00,87.0,473.0,156.0,5.0
2023-01-01 00:15:00+00:00,87.0,333.0,120.0,5.0
2023-01-01 00:30:00+00:00,87.0,328.0,108.0,5.0
2023-01-01 00:45:00+00:00,87.0,415.0,125.0,5.0
2023-01-01 01:00:00+00:00,87.0,341.0,110.0,5.0


In [204]:

pivot.head(5)

Type,humidity,pm10,pm25,temperature
Stamp,,,,
2023-01-01 00:00:00+00:00,87.0,473.0,156.0,5.0
2023-01-01 00:15:00+00:00,87.0,333.0,120.0,5.0
2023-01-01 00:30:00+00:00,87.0,328.0,108.0,5.0
2023-01-01 00:45:00+00:00,87.0,415.0,125.0,5.0
2023-01-01 01:00:00+00:00,87.0,341.0,110.0,5.0


In [205]:
pivot.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 66332 entries, 2023-01-01 00:00:00+00:00 to 2024-11-21 22:45:00+00:00
Freq: 15min
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   humidity     66332 non-null  float64
 1   pm10         66332 non-null  float64
 2   pm25         66332 non-null  float64
 3   temperature  66332 non-null  float64
dtypes: float64(4)
memory usage: 2.5 MB


In [206]:
#create lags for the past 3 days 
COLUMNS = pivot.columns
columns_for_lag=pivot.columns
for feature in columns_for_lag:
    for lag in range(1,4):
        pivot[f'{feature}_lag_{lag}']=pivot[feature].shift(lag)

In [207]:
pivot = pivot.dropna()

In [208]:
pivot.isnull().sum()

Type
humidity             0
pm10                 0
pm25                 0
temperature          0
humidity_lag_1       0
humidity_lag_2       0
humidity_lag_3       0
pm10_lag_1           0
pm10_lag_2           0
pm10_lag_3           0
pm25_lag_1           0
pm25_lag_2           0
pm25_lag_3           0
temperature_lag_1    0
temperature_lag_2    0
temperature_lag_3    0
dtype: int64

In [209]:
pivot.head(5)

Type,humidity,pm10,pm25,temperature,humidity_lag_1,humidity_lag_2,humidity_lag_3,pm10_lag_1,pm10_lag_2,pm10_lag_3,pm25_lag_1,pm25_lag_2,pm25_lag_3,temperature_lag_1,temperature_lag_2,temperature_lag_3
Stamp,,,,,,,,,,,,,,,,
2023-01-01 00:45:00+00:00,87.0,415.0,125.0,5.0,87.0,87.0,87.0,328.0,333.0,473.0,108.0,120.0,156.0,5.0,5.0,5.0
2023-01-01 01:00:00+00:00,87.0,341.0,110.0,5.0,87.0,87.0,87.0,415.0,328.0,333.0,125.0,108.0,120.0,5.0,5.0,5.0
2023-01-01 01:15:00+00:00,87.0,326.0,110.0,5.0,87.0,87.0,87.0,341.0,415.0,328.0,110.0,125.0,108.0,5.0,5.0,5.0
2023-01-01 01:30:00+00:00,87.0,330.0,110.0,5.0,87.0,87.0,87.0,326.0,341.0,415.0,110.0,110.0,125.0,5.0,5.0,5.0
2023-01-01 01:45:00+00:00,87.0,337.0,112.0,5.0,87.0,87.0,87.0,330.0,326.0,341.0,110.0,110.0,110.0,5.0,5.0,5.0


In [210]:
pivot.sort_index()

Type,humidity,pm10,pm25,temperature,humidity_lag_1,humidity_lag_2,humidity_lag_3,pm10_lag_1,pm10_lag_2,pm10_lag_3,pm25_lag_1,pm25_lag_2,pm25_lag_3,temperature_lag_1,temperature_lag_2,temperature_lag_3
Stamp,,,,,,,,,,,,,,,,
2023-01-01 00:45:00+00:00,87.0,415.0,125.0,5.0,87.0,87.0,87.0,328.0,333.0,473.0,108.0,120.0,156.0,5.0,5.0,5.0
2023-01-01 01:00:00+00:00,87.0,341.0,110.0,5.0,87.0,87.0,87.0,415.0,328.0,333.0,125.0,108.0,120.0,5.0,5.0,5.0
2023-01-01 01:15:00+00:00,87.0,326.0,110.0,5.0,87.0,87.0,87.0,341.0,415.0,328.0,110.0,125.0,108.0,5.0,5.0,5.0
2023-01-01 01:30:00+00:00,87.0,330.0,110.0,5.0,87.0,87.0,87.0,326.0,341.0,415.0,110.0,110.0,125.0,5.0,5.0,5.0
2023-01-01 01:45:00+00:00,87.0,337.0,112.0,5.0,87.0,87.0,87.0,330.0,326.0,341.0,110.0,110.0,110.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-21 21:45:00+00:00,74.0,108.0,47.0,2.0,74.0,73.0,71.0,107.0,115.0,119.0,49.0,52.0,49.0,2.0,3.0,3.0
2024-11-21 22:00:00+00:00,74.0,79.0,35.0,2.0,74.0,74.0,73.0,108.0,107.0,115.0,47.0,49.0,52.0,2.0,2.0,3.0
2024-11-21 22:15:00+00:00,74.0,81.0,33.0,2.0,74.0,74.0,74.0,79.0,108.0,107.0,35.0,47.0,49.0,2.0,2.0,2.0


In [211]:
pivot.sort_index(inplace=True)

In [212]:
from sklearn.model_selection import train_test_split
x = pivot.drop(columns=COLUMNS).head(1000)
y = pivot[COLUMNS].head(1000)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

In [213]:
y

Type,humidity,pm10,pm25,temperature
Stamp,,,,
2023-01-01 00:45:00+00:00,87.0,415.0,125.0,5.0
2023-01-01 01:00:00+00:00,87.0,341.0,110.0,5.0
2023-01-01 01:15:00+00:00,87.0,326.0,110.0,5.0
2023-01-01 01:30:00+00:00,87.0,330.0,110.0,5.0
2023-01-01 01:45:00+00:00,87.0,337.0,112.0,5.0
...,...,...,...,...
2023-01-11 09:30:00+00:00,77.0,30.0,18.0,7.0
2023-01-11 09:45:00+00:00,75.0,28.0,16.0,7.0
2023-01-11 10:00:00+00:00,73.0,4.0,2.0,8.0


In [245]:
#kreirame model RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100,random_state=42)
model.fit(X_train,y_train)


RandomForestRegressor(random_state=42)

In [247]:
y_pred = model.predict(X_test)

In [249]:
from sklearn.metrics import r2_score
print("R2-score: ",r2_score(y_test,y_pred))

R2-score:  0.16234927452866832


In [255]:
#kreirame xgboost model
from xgboost import XGBRegressor
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train,y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [257]:
xgb_pred = xgb_model.predict(X_test)

In [259]:
print("XGBRegressor r2-score: ",r2_score(y_test,xgb_pred))

XGBRegressor r2-score:  -0.1547802984714508


In [261]:
#BONUS 

In [263]:
#podelba 90:10
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False)

In [267]:
#fine tune na n_estimators
from sklearn.model_selection import GridSearchCV
param_grid={
    'n_estimators':[100,200,80],
    'random_state':[42]
}
grid_search = GridSearchCV(estimator=model,param_grid=param_grid,verbose=1,n_jobs=-1,scoring='r2')
grid_search.fit(X_train,y_train)
print("Best params: ",grid_search.best_params_)
print("Best score: ",grid_search.best_score_)


Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best params:  {'n_estimators': 80, 'random_state': 42}
Best score:  0.38866146937800183


In [ ]:
#ZAKLUCHOK pivot table koga pravime target promenlivite gledame vo koja kolona se naogjaat (vo ovoj sluchaj TYPE) columns=Type